In [1]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from moment.common import PATHS
from moment.utils.ucr_anomaly_archive_subset import ucr_anomaly_archive_subset

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
finetuning_mode = 'linear-probing'
experiment_name = "zero_shot_anomaly_detection"

results_path = os.path.join(PATHS.RESULTS_DIR, experiment_name)
print("Results path:", results_path)

In [ ]:
datasets_without_results = []
results = []
for dataset_name in tqdm(ucr_anomaly_archive_subset, total=len(ucr_anomaly_archive_subset)): 
    metadata = dataset_name.split('/')[-1].split('_')
    data_id, data_name = metadata[0], metadata[3]

    if not os.path.exists(os.path.join(results_path, f"results_{data_id}_{data_name}.csv")):
        datasets_without_results.append(dataset_name)
        continue
    
    best_result_path = os.path.join(results_path, f"results_{data_id}_{data_name}.csv")
    df = pd.read_csv(best_result_path, index_col=0).T
    df['Dataset Name'] = data_name
    df['Dataset ID'] = data_id

    last_modified_time = int(os.path.getmtime(best_result_path))
    last_modified_time = datetime.utcfromtimestamp(last_modified_time).strftime('%d-%m-%Y %H:%M')
    df['Timestamp'] = last_modified_time
    
    results.append(df)

results = pd.concat(results, axis=0).reset_index(drop=True)
results = results[[
    'Dataset ID', 'Dataset Name', 'Adj. Best F1', 
    'rAUCROC', 'rAUCPR', 'VUSROC', 'VUSPR', 'Timestamp']]

results.to_csv("../../assets/results/zero_shot/anomaly_detection.csv", index=False)

In [ ]:
results.dropna(inplace=True)
print(f"Results: {results.shape}" )
results.head()

In [ ]:
results[['Adj. Best F1', 'rAUCROC', 'rAUCPR', 'VUSROC', 'VUSPR']].describe()

In [ ]:
labels = ['Adj. Best F1', 'VUSROC']
data = results[labels].to_numpy().astype(np.float32)
labels = ['Adj. Best $F_1$', 'VUSROC']

boxprops = dict(linestyle='-', linewidth=1, color='k')
flierprops = dict(marker='o', markersize=12, markeredgecolor='darkgreen')
medianprops = dict(linestyle='-', linewidth=2, color='blue')
meanpointprops = dict(marker='D', markeredgecolor='black',
                      markerfacecolor='firebrick')
meanlineprops = dict(linestyle='--', linewidth=2, color='red')

_ = plt.boxplot(
    x=data, labels=labels, showmeans=True, 
    meanline=True, bootstrap=10000, notch=True, 
    boxprops=boxprops, flierprops=flierprops, 
    medianprops=medianprops, 
    meanprops=meanlineprops)

plt.ylabel("Accuracy", fontsize=16)
plt.xlabel("Metrics", fontsize=16)
plt.ylim(0.0, None)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(color='lightgray', linestyle='--', linewidth=0.5) 
plt.savefig("../../assets/figures/zero_shot/anomaly_detection.png", dpi=300, bbox_inches='tight')
plt.show()